In [13]:
%load_ext autoreload
%autoreload 2

import sys
import math
import time
import pickle
import argparse
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import csv
from sklearn import svm
import pandas as pd
import itertools
from itertools import zip_longest
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch
import torch.serialization


project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

from model.model_data_class import SimpleNN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# check for gpu
if torch.backends.mps.is_available():
    print("MPS is available")
    device = torch.device('mps')
else:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Using device:', device)

print('Using device:', device)

MPS is available
Using device: mps


In [7]:
input_size = 3
hidden_size = 20
output_size = 1
hidden_layers = 5

# Import your model class
from model.model_data_class import SimpleNN
model = SimpleNN(input_size, hidden_size, output_size, hidden_layers)


In [28]:
!ls ../spartan/jobs/model-files/job_4075299/

best-model.pth test-loss.pkl  train-loss.pkl


In [47]:

job_id = 4075299

# Construct the relative path to the model file
model_path = f'../spartan/jobs/model-files/job_{job_id}/'
model_file = os.path.join(model_path,"best-model.pth")

# Check if the file exists
if not os.path.exists(model_file):
    raise FileNotFound

state_dict = torch.load(model_file,weights_only=False,map_location=device)
model.load_state_dict(state_dict)
model.to(device)



SimpleNN(
  (base): Sequential(
    (0): Linear(in_features=3, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Tanh()
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): Tanh()
    (6): Linear(in_features=20, out_features=20, bias=True)
    (7): Tanh()
    (8): Linear(in_features=20, out_features=20, bias=True)
    (9): Tanh()
    (10): Linear(in_features=20, out_features=20, bias=True)
    (11): Tanh()
    (12): Linear(in_features=20, out_features=1, bias=True)
  )
)

In [ ]:
# Load the simulation data and then plo the model performance on the borders and on the inside region.

NameError: name 'test_loader' is not defined